In [1]:
import pandas as pd
import mysql.connector
import re
conn = mysql.connector.connect(username= "root",
                               host = "127.0.0.1",
                               password = "Admin@123",
                               database = "dataspark1")

cursor = conn.cursor()

In [2]:
df_product = pd.read_csv('Products.csv')

In [4]:
df_product.columns = df_product.columns.str.lower()
df_product.columns = df_product.columns.str.replace(" ","_")
df_product.columns

Index(['productkey', 'product_name', 'brand', 'color', 'unit_cost_usd',
       'unit_price_usd', 'subcategorykey', 'subcategory', 'categorykey',
       'category'],
      dtype='object')

In [5]:
df_product.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2517 entries, 0 to 2516
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   productkey      2517 non-null   int64 
 1   product_name    2517 non-null   object
 2   brand           2517 non-null   object
 3   color           2517 non-null   object
 4   unit_cost_usd   2517 non-null   object
 5   unit_price_usd  2517 non-null   object
 6   subcategorykey  2517 non-null   int64 
 7   subcategory     2517 non-null   object
 8   categorykey     2517 non-null   int64 
 9   category        2517 non-null   object
dtypes: int64(3), object(7)
memory usage: 196.8+ KB


In [8]:
def extract_numeric(value):
    return float(re.sub(r'[^\d.]', '', value))

# Apply function to the relevant columns
df_product['unit_cost_usd'] = df_product['unit_cost_usd'].apply(extract_numeric)
df_product['unit_price_usd'] = df_product['unit_price_usd'].apply(extract_numeric)

In [10]:
df_product.head()

,productkey,product_name,brand,color,unit_cost_usd,unit_price_usd,subcategorykey,subcategory,categorykey,category
0,1,Contoso 512MB MP3 Player E51 Silver,Contoso,Silver,6.62,12.99,101,MP4&MP3,1,Audio
1,2,Contoso 512MB MP3 Player E51 Blue,Contoso,Blue,6.62,12.99,101,MP4&MP3,1,Audio
2,3,Contoso 1G MP3 Player E100 White,Contoso,White,7.40,14.52,101,MP4&MP3,1,Audio
3,4,Contoso 2G MP3 Player E200 Silver,Contoso,Silver,11.00,21.57,101,MP4&MP3,1,Audio
4,5,Contoso 2G MP3 Player E200 Red,Contoso,Red,11.00,21.57,101,MP4&MP3,1,Audio


In [11]:
df_product.shape

(2517, 10)

In [13]:
insert_product_values = '''
    INSERT INTO product (
        productkey, 
        product_name, 
        brand, 
        color, 
        unit_cost_usd, 
        unit_price_usd, 
        subcategorykey, 
        subcategory, 
        categorykey, 
        category
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
'''

# Insert data into the product table
for index, row in df_product.iterrows():
    cursor.execute(insert_product_values, (
        row['productkey'],
        row['product_name'],
        row['brand'],
        row['color'],
        row['unit_cost_usd'],
        row['unit_price_usd'],
        row['subcategorykey'],
        row['subcategory'],
        row['categorykey'],
        row['category']
    ))
conn.commit()